In [2]:
import pandas as pd

In [3]:
data = pd.read_csv("爬取的典故数据.csv",sep = "	")
data.head()

,sentence,author,title,allusion,variation_number,allusion_index
0,浮世度千载，桃源方一春。,于武陵,赠王道士,桃源（陶潜）,1,"[[6, 7]]"
1,曾逢异人说，风景似桃源。,于鹄,南溪书斋,桃源（陶潜）,1,"[[9, 10]]"
2,不言沁园好，独隐武陵花。,储光羲,玉真公主山居,桃源（陶潜）,1,"[[8, 9]]"
3,鸡犬疑沾药，耕桑似避秦。,储嗣宗,春游望仙谷,桃源（陶潜）,1,"[[9, 10]]"
4,僻居谋道不谋身，避病桃源不避秦。,刘商,袁十五远访山门,桃源（陶潜）,2,"[[10, 11], [13, 14]]"


In [4]:
data["sentence"].value_counts()

绣衣柱史何昂藏，铁冠白笔横秋霜。        4
不如何逊无佳句，若比冯唐是壮年。        4
如今岂无騕袅与骅骝，时无王良伯乐死即休。    4
自叹马卿常带病，还嗟李广未封侯。        4
倚玉甘无路，穿杨却未期。            4
                       ..
萤聚帐中人已去，鹤离台上月空圆。        1
勤苦流萤信，吁嗟宿燕知。            1
皇明如照隐，愿及聚萤人。            1
毕事驱传马，安居守窗萤。            1
颂椒添讽咏，禁火卜欢娱。            1
Name: sentence, Length: 13393, dtype: int64

In [5]:
data[data["sentence"] == "绣衣柱史何昂藏，铁冠白笔横秋霜。"]

,sentence,author,title,allusion,variation_number,allusion_index
3158,绣衣柱史何昂藏，铁冠白笔横秋霜。,李白,赠潘侍御论钱少阳,绣衣,1,"[[0, 1]]"
8268,绣衣柱史何昂藏，铁冠白笔横秋霜。,李白,赠潘侍御论钱少阳,柱史（张苍）,1,"[[2, 3]]"
10720,绣衣柱史何昂藏，铁冠白笔横秋霜。,李白,赠潘侍御论钱少阳,铁冠,1,"[[8, 9]]"
14359,绣衣柱史何昂藏，铁冠白笔横秋霜。,李白,赠潘侍御论钱少阳,白笔,1,"[[10, 11]]"


按照 sentence 分组并合并其他列

In [6]:
result = data.groupby('sentence').agg({
    'author': 'first',  # 保留第一条作者
    'title': 'first',   # 保留第一条标题
    'allusion': lambda x: ';'.join(x),  # 合并典故，以分号分隔
    'variation_number': 'sum',  # 求和典故变体数
    'allusion_index': lambda x: ';'.join(x)  # 合并索引，逗号分隔后再用分号分隔不同典故
}).reset_index()


result[result["sentence"] == "绣衣柱史何昂藏，铁冠白笔横秋霜。"]

,sentence,author,title,allusion,variation_number,allusion_index
9816,绣衣柱史何昂藏，铁冠白笔横秋霜。,李白,赠潘侍御论钱少阳,绣衣;柱史（张苍）;铁冠;白笔,4,"[[0, 1]];[[2, 3]];[[8, 9]];[[10, 11]]"


调整allusion保存格式

In [7]:
def transform_allusion_index(row):
    # 拆分 allusion 和 allusion_index
    allusions = row['allusion'].split(';')
    allusion_indices = row['allusion_index'].split(';')
    
    # 组合为目标格式
    combined = []
    for allusion, indices in zip(allusions, allusion_indices):
        index_list = indices.strip('[]')  # 去掉索引的外部括号
        combined.append(f"[{index_list},{allusion}]")  # 按目标格式拼接

    return ';'.join(combined)

# 对 DataFrame 的每一行应用函数
result['transformed_allusion'] = result.apply(transform_allusion_index, axis=1)

# 查看结果
result[result["sentence"] == "绣衣柱史何昂藏，铁冠白笔横秋霜。"]

,sentence,author,title,allusion,variation_number,allusion_index,transformed_allusion
9816,绣衣柱史何昂藏，铁冠白笔横秋霜。,李白,赠潘侍御论钱少阳,绣衣;柱史（张苍）;铁冠;白笔,4,"[[0, 1]];[[2, 3]];[[8, 9]];[[10, 11]]","[0, 1,绣衣];[2, 3,柱史（张苍）];[8, 9,铁冠];[10, 11,白笔]"


In [8]:
result.head()

,sentence,author,title,allusion,variation_number,allusion_index,transformed_allusion
0,㪷酒上河梁，惊魂去越乡。,武元衡,送寇侍御司马之明州,河梁,1,"[[3, 4]]","[3, 4,河梁]"
1,㸌如羿射九日落，矫如群帝骖龙翔。,杜甫,观公孙大娘弟子舞剑器行,射日弓（后羿）,2,"[[4, 5, 6], [2, 3, 4, 5]]","[4, 5, 6], [2, 3, 4, 5,射日弓（后羿）]"
2,䌽笔谕戎矜倚马，华堂留客看惊鸿。,刘禹锡,酬窦员外郡斋宴客偶命柘枝因见寄兼呈张十一院长元九侍御,倚马（桓温，袁虎）,1,"[[5, 6]]","[5, 6,倚马（桓温，袁虎）]"
3,䌽衣人竞看，银诏帝亲书。,卢纶,送浑炼归觐却赴阙庭,老莱（老莱子）,0,[],"[,老莱（老莱子）]"
4,一与清风上芸阁，再期秋雨过龙津。,鲍溶,寄张十七校书李仁行秀才,芸阁,1,"[[5, 6]]","[5, 6,芸阁]"


In [9]:
def adjust_transformed_allusion(row):
    # 获取 transformed_allusion 的值
    transformed_allusion = row['transformed_allusion']
    allusions = row['allusion'].split(';')  # 获取所有典故名列表
    
    # 初始化变量
    adjusted = transformed_allusion
    current_allusion_index = 0  # 当前典故的索引
    
    # 遍历 transformed_allusion 并找到 ] 后的 ,
    i = 0
    while i < len(adjusted):
        if adjusted[i] == ']' and i + 1 < len(adjusted) and adjusted[i + 1] == ',':
            # 在 ] 前插入当前典故名
            current_allusion = allusions[current_allusion_index]
            adjusted = adjusted[:i] + f',{current_allusion}' + adjusted[i:]
            i += len(current_allusion) + 1  # 跳过插入的典故名长度
            
            # 如果当前典故名已经完全应用，切换到下一个典故
            current_allusion_index = min(current_allusion_index + 1, len(allusions) - 1)
        
        i += 1
    
    return adjusted

# 对 DataFrame 的每一行应用函数
result['transformed_allusion'] = result.apply(adjust_transformed_allusion, axis=1)


def replace_comma_after_bracket(row):
    return row['transformed_allusion'].replace('],', '];')

# 对 `transformed_allusion` 列应用替换函数
result['transformed_allusion'] = result.apply(replace_comma_after_bracket, axis=1)


result.head()

,sentence,author,title,allusion,variation_number,allusion_index,transformed_allusion
0,㪷酒上河梁，惊魂去越乡。,武元衡,送寇侍御司马之明州,河梁,1,"[[3, 4]]","[3, 4,河梁]"
1,㸌如羿射九日落，矫如群帝骖龙翔。,杜甫,观公孙大娘弟子舞剑器行,射日弓（后羿）,2,"[[4, 5, 6], [2, 3, 4, 5]]","[4, 5, 6,射日弓（后羿）]; [2, 3, 4, 5,射日弓（后羿）]"
2,䌽笔谕戎矜倚马，华堂留客看惊鸿。,刘禹锡,酬窦员外郡斋宴客偶命柘枝因见寄兼呈张十一院长元九侍御,倚马（桓温，袁虎）,1,"[[5, 6]]","[5, 6,倚马（桓温，袁虎）]"
3,䌽衣人竞看，银诏帝亲书。,卢纶,送浑炼归觐却赴阙庭,老莱（老莱子）,0,[],"[,老莱（老莱子）]"
4,一与清风上芸阁，再期秋雨过龙津。,鲍溶,寄张十七校书李仁行秀才,芸阁,1,"[[5, 6]]","[5, 6,芸阁]"


结果：allusion以[4, 5, 6,射日弓（后羿）]; [2, 3, 4, 5,射日弓（后羿）]格式保存，每个保存以;分隔。

In [10]:
result.to_csv('data/final_data.csv', sep='\t', index=False, encoding='utf-8')